# package import & env setting

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI


import os
from dotenv import load_dotenv
load_dotenv()



os.environ["OPENWEATHER_API_KEY"] = os.getenv("OPENWEATHER_API_KEY")
model = ChatOpenAI(model="gpt-4o")

# MCP Client sse test

`python -m mcp.weatehr_mcp` 명령어로 sse server를 실행해준 상태로 아래 코드를 실행해야함!

In [2]:
weather_server_info = {
        "weather": {
            "url": "http://localhost:7000/sse",
            "transport": "sse",
        }
    }
async with MultiServerMCPClient(weather_server_info) as client:
    weather_tools = client.get_tools()
    agent = create_react_agent(model, client.get_tools())
    weather_response = await agent.ainvoke({"messages": "서울 날씨 알려줄래?"})
    print(weather_response['messages'][-1].content)

현재 서울의 날씨는 다음과 같습니다:

- 온도: 25.14도
- 날씨: 흐림
- 풍속: 2.67m/s

이러한 날씨에는 가벼운 긴팔 옷이나 얇은 가디건을 입는 것이 좋습니다. 바람이 조금 불 수 있으니 가벼운 외투를 챙기는 것도 추천드립니다.


In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class AgentState(TypedDict):
    input: str
    response: str
    route: str


def goto_weather(state: AgentState) -> AgentState:
    state["route"] = "weather"
    return state

async def make_graph(state: AgentState) -> AgentState:
    async with MultiServerMCPClient(weather_server_info) as client:
        agent = create_react_agent(model, client.get_tools())

        user_input = state["input"]
        weather_response = await agent.ainvoke({"messages": user_input})
        state["response"] = weather_response['messages'][-1].content
        return state

# 그래프 생성
def create_graph():
    graph = StateGraph(AgentState)
    
    graph.add_node("category", goto_weather)
    graph.add_node("weather", make_graph)

    graph.set_entry_point("category")
    graph.add_edge("category", "weather")
    graph.add_edge("weather", END)
    
    return graph.compile()

In [4]:
async def run_graph(user_input: str):
    graph = create_graph()
    result = await graph.ainvoke({"input": user_input,
                                  "response": "",
                                  "route": ""})
    return result

In [5]:
result = await run_graph("서울 날씨 알려줘")
print(result['response'])

현재 서울의 날씨는 다음과 같습니다:

- **시간**: 16:25
- **온도**: 23.76도
- **날씨**: 맑음
- **풍속**: 3.09m/s

오늘의 맑고 온도의 정도를 고려하면 가벼운 옷차림을 추천드립니다. 반팔 티셔츠나 셔츠와 청바지, 그리고 얇은 자켓을 가지고 다니시면 좋을 것 같아요. 바람이 조금 있지만 크게 불지 않으므로 걷기에 좋은 날씨입니다.
